In [1]:
# import sys
# !conda install --yes --prefix {sys.prefix} scikit-learn

In [2]:
import numpy as np
import pandas as pd
import folium
import webbrowser
import os
import math

from h3 import h3
from folium import Map

In [3]:
def load_data():
    foodbanks = pd.read_csv("~/Downloads/TT_Food_Bank_Locations_copy.csv")
    return foodbanks

In [4]:
def create_map(clusters):
    # Create the map object
    map = Map(tiles="cartodbpositron")
    
#     map = Map(tiles="cartodbdark_matter")              #alternative map type
    # map = Map(tiles="openstreetmap")                   #alternative map type

    # Convert the clusters dictionary items to polygons and add them to the map
    for cluster in clusters.values():
        points = cluster['geom']
        # points = [p[::-1] for p in points]
        if cluster['count'] == 1:
            tooltip = "{0} food bank".format(cluster['count'])     # when hover an area, displays number of food banks
        else:
            tooltip = "{0} food banks".format(cluster['count'])     # when hover an area, displays number of food banks

        polygon = folium.vector_layers.Polygon(locations=points, tooltip=tooltip,
                                               fill=True, 
                                               color='#ff0000', 
                                               fill_color='#ff0000', 
                                               fill_opacity=0.4, weight=3, opacity=0.4)
        polygon.add_to(map)

    # Determine the map bounding box
    max_lat = df.lat.max()
    min_lat = df.lat.min()
    max_lon = df.lng.max()
    min_lon = df.lng.min()
    
    # Fit the map to the bounds
    map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    
    return map

In [5]:
def show_map(map, file_name):
    map.save(file_name)
    wb = webbrowser.open('file://' + os.path.realpath(file_name), new=2)     #opens in new tab. using a new of 1 opens in a new window

In [6]:
# load_data function object from above is assigned to f_bank
f_bank = load_data()

In [7]:
f_bank.dropna()

# ensure only valid values for lat and long are used
f_bank = f_bank.loc[f_bank.lat <=  90.0]
f_bank = f_bank.loc[f_bank.lat >= -90.0]

f_bank = f_bank.loc[f_bank.lng <=  180.0]
f_bank = f_bank.loc[f_bank.lng >= -180.0]

In [8]:
h3_level = 6           # best values to use: 4,5,6,7


In [9]:
# iterates through each number in the individual conlumns of the dataset and converts them to an h3 index
def lat_lng_to_h3(row):
    return h3.geo_to_h3(row['lat'], row['lng'], h3_level)

In [10]:
# adds new column with h3 index to dataset
f_bank['h3_index_rep'] = f_bank.apply(lat_lng_to_h3, axis=1)

In [11]:
# copy data and save as df to avoid corrupting dataset
df = f_bank.copy()

In [12]:
# df.head(n=15)
df.iloc[260:267]

,Foodbank Name,Distribution Centre Name,Town,County,Postcode,lat,lng,h3_index_rep
260,Cannock & District,Penkridge Methodist Church,Penkridge,Staffordshire,ST19 5AH,52.7239,-2.1157,86195c18fffffff
261,Cannock & District,Portacabin - Sandwich Bar - CLOSED,NaN,NaN,WS11 1JR,52.6909,-2.0295,86195c037ffffff
262,Cannock & District,Stafford CAB - CLOSED,Stafford,Staffordshire,ST16 2LZ,52.8066,-2.1139,86195c1a7ffffff
263,Cannock & District,Coniston Hall - Collection/Delivery,Chadsmoor,Staffordshire,WS11 5HG,52.7032,-2.0152,86195c037ffffff
264,Cardiff,FBC Ely,Cardiff,NaN,CF5 4FN,51.4820,-3.2503,86195ab4fffffff
265,Cardiff,FBC Central,Cardiff,NaN,CF11 9AD,51.4813,-3.1888,8619586b7ffffff
266,Cardiff,FBC Cathays,Cardiff,NaN,CF24 4ND,51.4949,-3.1773,86195879fffffff


In [13]:
# find neighbouring hexagons for a given h3 index

# h3.k_ring('8a1909ac1867fff', 1)  # kring gives the index for the ring given in function and the (most likely) 6 surrounding hexagons
# h3.hex_ring('861909ac7ffffff', 1) # same as abov but does not include the origin hexagon, thus, hollow ring

In [14]:
h3_clusters = dict()

for index, row in df.iterrows():
    key = row['h3_index_rep']
    if key in h3_clusters:
        h3_clusters[key]["count"] += 1
    else:
        h3_clusters[key] = {"count": 1,
                            "geom": h3.h3_to_geo_boundary(key)}

In [15]:
relevant_clusters = { key:value for (key,value) in h3_clusters.items()}

In [16]:
h3_map = create_map(relevant_clusters)

In [17]:
show_map(h3_map, "h3_level:_{}_.html".format(h3_level))

In [18]:
# place all postcodes in the 'Postcodes' column of the dataframe into a list
column_as_series = (list(df['Postcode']))

# This function checks if the postcode in in the list above and return the h3 index of the postcode given 
def get_index_from_pc(postcode):
    if (postcode.upper()) in column_as_series:
        
        # if the postcode is a valid one and is in the dataframe then return its equivalent h3 index
        the_index = df.loc[(df['Postcode'] == (postcode.upper())), 'h3_index_rep'].iloc[0]  
        print('H3 index for the postcode provided is: {}'.format(the_index))
        
    else:
        print('There is no foodbank associated with the postcode you have given')



In [19]:
get_index_from_pc('CF24 4ND')

H3 index for the postcode provided is: 86195879fffffff


In [20]:
# place all indexes in the 'h3_index_rep' column of the dataframe into a list
column2_as_series = list(df['h3_index_rep'])

# This function checks if the postcode in in the list above and return the h3 index of the postcode given 
def get_pc_from_index(h3index):
    if (h3index.lower()) in column2_as_series:
        # if the postcode is a valid one and is in the dataframe then return its equivalent h3 index
        the_pc = df.loc[(df['h3_index_rep'] == (h3index.lower())), 'Postcode'].iloc[0]  
        print('Postcode for the H3 index provided is: {}'.format(the_pc))
    else:
        print('There is no foodbank associated with the index you have given')

In [21]:
get_pc_from_index('831958fffffffff')


There is no foodbank associated with the index you have given
